<a href="https://colab.research.google.com/github/i-hdk/pathing-colab/blob/main/pathing_bezier_to_desired_lin_%26_ang_vel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import to bezier point array

In [ ]:
import numpy as np
import os

line_count = sum(1 for _ in open('test.txt'))

print(f"The number of lines in test.txt is: {line_count}")
bezier = np.float64(np.arange(line_count*8).reshape(line_count,4,2))
l = 0
try:
  with open("test.txt", 'r') as file:
    for line in file:
      # Remove parentheses and split the line
      numbers = line.replace('(', '').replace(')', '').split(',')

      # Clean up the numbers
      cleaned_numbers = []
      for num in numbers:
          num = num.strip()
          if num:
              cleaned_numbers.extend([float(x) for x in num.split()])

      # Print the extracted numbers
      print(cleaned_numbers)
      bezier[l][0][0] = cleaned_numbers[0]
      bezier[l][0][1] = cleaned_numbers[1]
      bezier[l][1][0] = cleaned_numbers[2]
      bezier[l][1][1] = cleaned_numbers[3]
      bezier[l][2][0] = cleaned_numbers[4]
      bezier[l][2][1] = cleaned_numbers[5]
      bezier[l][3][0] = cleaned_numbers[6]
      bezier[l][3][1] = cleaned_numbers[7]
      l+=1

except FileNotFoundError:
    print("The file does not exist.")

print(bezier)


The number of lines in test.txt is: 2
[82.14, 9.67, 88.83, 9.43, 98.77, 7.5, 104.85, 11.34]
[104.85, 11.34, 109.39, 14.21, 105.81, 22.35, 108.2, 35.28]
[[[ 82.14   9.67]
  [ 88.83   9.43]
  [ 98.77   7.5 ]
  [104.85  11.34]]

 [[104.85  11.34]
  [109.39  14.21]
  [105.81  22.35]
  [108.2   35.28]]]


# reversed import

In [ ]:
import numpy as np
import os

line_count = sum(1 for _ in open('test.txt'))

print(f"The number of lines in test.txt is: {line_count}")
bezier = np.float64(np.arange(line_count*8).reshape(line_count,4,2))
l = 0
try:
  with open("test.txt", 'r') as file:
    for line in file:
      # Remove parentheses and split the line
      numbers = line.replace('(', '').replace(')', '').split(',')

      # Clean up the numbers
      cleaned_numbers = []
      for num in numbers:
          num = num.strip()
          if num:
              cleaned_numbers.extend([float(x) for x in num.split()])

      # Print the extracted numbers
      print(cleaned_numbers)
      bezier[l][0][0] = cleaned_numbers[0]
      bezier[l][0][1] = cleaned_numbers[1]
      bezier[l][1][0] = cleaned_numbers[2]
      bezier[l][1][1] = cleaned_numbers[3]
      bezier[l][2][0] = cleaned_numbers[4]
      bezier[l][2][1] = cleaned_numbers[5]
      bezier[l][3][0] = cleaned_numbers[6]
      bezier[l][3][1] = cleaned_numbers[7]
      l+=1

except FileNotFoundError:
    print("The file does not exist.")

print(bezier)


The number of lines in test.txt is: 2
[82.14, 9.67, 88.83, 9.43, 98.77, 7.5, 104.85, 11.34]
[104.85, 11.34, 109.39, 14.21, 105.81, 22.35, 108.2, 35.28]
[[[ 82.14   9.67]
  [ 88.83   9.43]
  [ 98.77   7.5 ]
  [104.85  11.34]]

 [[104.85  11.34]
  [109.39  14.21]
  [105.81  22.35]
  [108.2   35.28]]]


# calculate velocities

In [ ]:
import math

def bezier_value(p0, p1, p2, p3, t): #ps should be scalars, like the x or y component of each point
  term1 = ((1-t)**3)*p0
  term2 = 3*((1-t)**2)*t*p1
  term3 = 3*((1-t))*(t**2)*p2
  term4 = (t**3)*p3
  return term1 + term2 + term3 + term4

def bezier_derivative(p0, p1, p2, p3, t):
  term1 = 3*((1-t)**2)*(p1-p0)
  term2 = 6*(1-t)*t*(p2-p1)
  term3 = 3*(t**2)*(p3-p2)
  return term1 + term2 + term3

def bezier_acceleration(p0, p1, p2, p3, t):
  term1 = 6*(1-t)*(p2-2*p1+p0)
  term2 = 6*t*(p3-2*p2+p1)
  return term1 + term2

def arc_length(p0x,p0y,p1x,p1y,p2x,p2y,p3x,p3y,a,b):
  x = 0
  start = a
  stop = b
  step = integration_interval_width
  intervals = np.linspace(start, stop, int((stop - start) / step))
  for i in intervals:
    integrand = math.sqrt((bezier_derivative (p0x,p1x,p2x,p3x,i))**2+(bezier_derivative(p0y,p1y,p2y,p3y,i))**2)
    x+=integrand*integration_interval_width
  return x

def curvature(p0x,p0y,p1x,p1y,p2x,p2y,p3x,p3y,t):
  bax = bezier_acceleration(p0x,p1x,p2x,p3x,t)
  bdy = bezier_derivative(p0y,p1y,p2y,p3y,t)
  bay = bezier_acceleration(p0y,p1y,p2y,p3y,t)
  bdx = bezier_derivative(p0x,p1x,p2x,p3x,t)
  numerator = bax*bdy-bdx*bay
  denominator = (bdx**2+bdy**2)**1.5
  return numerator/denominator

#inputs
desired_speed = 10 #inches/second
max_acceleration = 5 #inches/second^2
integration_interval_width = 0.000001 #used for estimating integrals for arc length of parametric curve
predicted_time = 20 #seconds of path, need to overestimate or else not enough space for output array
bezier_splits = 60000

desired_speed_ms = desired_speed/1000
time_to_max_ms = math.ceil(desired_speed/max_acceleration*1000)
output = np.arange(1000*predicted_time*7)
output = np.float64(output.reshape(1000*predicted_time,7)) #output should return for every ms, lin & angular velocity, bezier count & t*1000
bezier_ms = np.arange(len(bezier)*bezier_splits) #gives ms in output array based on t and which bezier
bezier_ms = bezier_ms.reshape(len(bezier),bezier_splits)

dee = 0
ms_count = 0 #used to start at 1
temp_arc_length = 0
setpoint = 0
for b in range(len(bezier)):
  for i in range(bezier_splits):
    temp_arc_length+=arc_length(bezier[b][0][0],bezier[b][0][1],bezier[b][1][0],bezier[b][1][1],bezier[b][2][0],bezier[b][2][1],bezier[b][3][0],bezier[b][3][1],i/bezier_splits,(i+1)/bezier_splits)
    if ms_count < time_to_max_ms:
      setpoint = ms_count/time_to_max_ms*desired_speed_ms
    else:
      setpoint = desired_speed_ms
      """
    if i > 500 and i < 1000:
      print("temp_arc_length: " + str(temp_arc_length))
      print("setpoint: " + str(setpoint))
      """

    if temp_arc_length >= setpoint:
      output[ms_count][0] = setpoint*1000 #inch/sec
      radius = 1/curvature(bezier[b][0][0],bezier[b][0][1],bezier[b][1][0],bezier[b][1][1],bezier[b][2][0],bezier[b][2][1],bezier[b][3][0],bezier[b][3][1],i/bezier_splits)
      output[ms_count][1] = output[ms_count][0]/radius #radians/sec
      output[ms_count][2] = b
      output[ms_count][3] = i
      if dee < 2:
        dee = dee + 1
        print(i)
        print(ms_count)
      if dee == 2:
        print(bezier_value(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits))
        print(bezier_value(bezier[b][0][1],bezier[b][1][1],bezier[b][2][1],bezier[b][3][1],i/bezier_splits))
        dee = dee + 1
      output[ms_count][4] = bezier_value(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits)
      output[ms_count][5] = bezier_value(bezier[b][0][1],bezier[b][1][1],bezier[b][2][1],bezier[b][3][1],i/bezier_splits)
      if dee == 3:
        print(output[ms_count][4])
        print(output[ms_count][5])
        dee = dee + 1
      if bezier_derivative(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits) == 0:
        output[ms_count][6] = 0
      else:
        output[ms_count][6] = math.pi/2.0 - math.atan2(bezier_derivative(bezier[b][0][1],bezier[b][1][1],bezier[b][2][1],bezier[b][3][1],i/bezier_splits),bezier_derivative(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits))
        while output[ms_count][6] < -math.pi:
          output[ms_count][6] = output[ms_count][6] + math.pi
        while output[ms_count][6] > math.pi:
          output[ms_count][6] = output[ms_count][6] - math.pi

      temp_arc_length = 0
      ms_count+=1
    bezier_ms[b][i] = ms_count-1 #maybe take away the -1 close enough?? not sure if this throws away accuracy
print(ms_count)
for i in range (ms_count,len(output),1):
  #print("hi")
  output[i][0] = 0

print(output[1][4])
print(output[1][5])
#go from backwards
temp_arc_length = 0
backwards_ms_count = 1
appended_output = np.float64(np.arange(time_to_max_ms*7).reshape(time_to_max_ms,7))
append_ms = 0
inside_break = 0

for b in range(len(bezier)-1, -1, -1):
    for i in range(bezier_splits-1,-1,-1):
      temp_arc_length+=arc_length(bezier[b][0][0],bezier[b][0][1],bezier[b][1][0],bezier[b][1][1],bezier[b][2][0],bezier[b][2][1],bezier[b][3][0],bezier[b][3][1],i/bezier_splits,(i+1)/bezier_splits)
      if backwards_ms_count < time_to_max_ms:
        setpoint = backwards_ms_count/time_to_max_ms*desired_speed_ms
      else:
        setpoint = desired_speed_ms
      #if i > 500 and i < 1000:
       # print("temp_arc_length: " + str(temp_arc_length))
        #print("setpoint: " + str(setpoint))
      if temp_arc_length >= setpoint:
        appended_output[backwards_ms_count][0] = setpoint*1000 #inch/sec
        radius = 1/curvature(bezier[b][0][0],bezier[b][0][1],bezier[b][1][0],bezier[b][1][1],bezier[b][2][0],bezier[b][2][1],bezier[b][3][0],bezier[b][3][1],i/bezier_splits)
        appended_output[backwards_ms_count][1] = output[backwards_ms_count][0]/radius #radians/sec
        appended_output[backwards_ms_count][2] = b
        appended_output[backwards_ms_count][3] = i
        appended_output[backwards_ms_count][4] = bezier_value(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits)
        appended_output[backwards_ms_count][5] = bezier_value(bezier[b][0][1],bezier[b][1][1],bezier[b][2][1],bezier[b][3][1],i/bezier_splits)
        if bezier_derivative(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits) == 0:
          appended_output[backwards_ms_count][6] = 0
        else:
          appended_output[backwards_ms_count][6] = math.pi/2.0 - math.atan2(bezier_derivative(bezier[b][0][1],bezier[b][1][1],bezier[b][2][1],bezier[b][3][1],i/bezier_splits),bezier_derivative(bezier[b][0][0],bezier[b][1][0],bezier[b][2][0],bezier[b][3][0],i/bezier_splits))
          while appended_output[backwards_ms_count][6] < -math.pi:
            appended_output[backwards_ms_count][6] = appended_output[backwards_ms_count][6] + math.pi
          while appended_output[backwards_ms_count][6] > math.pi:
            appended_output[backwards_ms_count][6] = appended_output[backwards_ms_count][6] - math.pi
        temp_arc_length = 0
        backwards_ms_count+=1
        if backwards_ms_count>=time_to_max_ms:
          append_ms = bezier_ms[b][i]+1 #careful, might hv to attach on the break outside of this for loop if theres multiple bezier curves
          break;
        #print("setpoint: " + str(setpoint*1000))
        #print(output[bezier_ms[b][i]][0])
        if setpoint*1000 >= output[bezier_ms[b][i]][0]:
          print("setpoint: " + str(setpoint*1000))
          print(output[bezier_ms[b][i]][0])
          append_ms = bezier_ms[b][i]+1
          print("break called")
          break;
    if backwards_ms_count>=time_to_max_ms:
           break;
print("append_ms: " + str(append_ms))
print(output[1][4])
print(output[1][5])
for m in range (append_ms, append_ms+backwards_ms_count-1,1):
  output[m][0] = appended_output[backwards_ms_count-1+append_ms-m][0]
  output[m][1] = appended_output[backwards_ms_count-1+append_ms-m][1]
  output[m][2] = appended_output[backwards_ms_count-1+append_ms-m][2]
  output[m][3] = appended_output[backwards_ms_count-1+append_ms-m][3]
  output[m][4] = appended_output[backwards_ms_count-1+append_ms-m][4]
  output[m][5] = appended_output[backwards_ms_count-1+append_ms-m][5]
  output[m][6] = appended_output[backwards_ms_count-1+append_ms-m][6]

print(output[1][4])
print(output[1][5])

for i in range(append_ms+backwards_ms_count-1,len(output),1):
  output[i][0] = 0
  output[i][1] = 0

output_file = os.path.join('/content/', 'output.txt')
np.savetxt(output_file, output)

#todo: decceleration part is int, not float
print(output[1][4])
print(output[1][5])

0
0
1
1
82.1403345027083
9.669987998591703
82.1403345027083
9.669987998591703
5493
82.1403345027083
9.669987998591703
append_ms: 4464
82.1403345027083
9.669987998591703
82.1403345027083
9.669987998591703
82.1403345027083
9.669987998591703


# debugging




In [ ]:
print(bezier_value(bezier[0][0][0],bezier[0][1][0],bezier[0][2][0],bezier[0][3][0],0))
print(bezier_value(82.14,88.83,98.77,104.85,1/60000))
print(bezier_value(82.14,88.83,98.77,104.85,1/60000))

82.14
82.1403345027083
